# Making Colored Videos

Imports

In [ ]:
# Visuals
import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import pyplot as plt
import seaborn as sns

# Generic
import numpy as np
import cupy

# I/O
from imaging.IO import load_single_tiff, save_video, save_single_tiff

# Image Processing
from imaging.ImageProcessing import fast_filter_images, grouped_z_project

# Coloring
from imaging.Coloring import ColorImages, rescale_images, colorize_rois, generate_custom_map, generate_pixel_pairs
from imaging.Colorizer import merge_background, overlay_colorized_rois

Load Data

In [ ]:
images = load_single_tiff("C:\\Users\\YUSTE\\Desktop\\UpDown.tif", 5600)
stat = np.load("C:\\Users\\YUSTE\\Desktop\\AA\\data\\suite2p\\plane0\\stat.npy", allow_pickle=True)
iscell = np.load("C:\\Users\\YUSTE\\Desktop\\AA\\data\\suite2p\\plane0\\iscell.npy", allow_pickle=True)

Beautify

In [ ]:
# images = fast_filter_images(images, Footprint=np.ones((7, 1, 1))).get() # Get is simply to convert from cupy array

In [ ]:
# save_single_tiff(images, "C:\\Users\\YUSTE\\Desktop\\AA_01_17_23_K.tif", np.int16)

Select Colors

In [ ]:
blue = (0.0745, 0.6234, 1.000)
red = (1.000, 0.2941, 0.3068)

Subset Neurons

In [ ]:
subset_neurons_1 = \
    [34, 37, 42, 49, 52, 54, 59, 61, 65, 66, 84, 91, 117, 129, 138, 143, 146, 162, 176, 177, 181, 185, 217]
subset_neurons_1 = [neuron-1 for neuron in subset_neurons_1] # adj for matlab non-zero indexing

subset_neurons_2 = [24, 78]
subset_neurons_2 = [neuron-1 for neuron in subset_neurons_2] # adj for matlab non-zero indexing

In [ ]:
ColorVideo = ColorImages(images, stat, iscell)
ColorVideo.cutoffs = (1.0, 99.75)
ColorVideo.background_style = "True"
ColorVideo.background_cutoffs = (5.0, 99.99)
ColorVideo.neuron_subsets = ([ColorVideo.neuronal_ids[neuron] for neuron in subset_neurons_1], blue)
ColorVideo.neuron_subsets = ([ColorVideo.neuronal_ids[neuron] for neuron in subset_neurons_2], red)

Generate custom colormaps

In [ ]:
base = (0.2073382588235294, 0.2073382588235294, 0.2073382588235294)
_cmap1 = generate_custom_map([base, base, blue, blue])
_cmap2 = generate_custom_map([base, base, red, red])

Rescale images using cutoffs

In [ ]:
scaled_image = rescale_images(images, *ColorVideo.cutoffs)

Colorize ROIs

In [ ]:
CR1 = colorize_rois(scaled_image.copy(), stat, ColorVideo.neuron_subsets[0], _cmap1)
CR2 = colorize_rois(scaled_image.copy(), stat, ColorVideo.neuron_subsets[1], _cmap2)

Extract background

In [ ]:
background_1 = ColorVideo.background.copy()
background_2 = ColorVideo.background.copy()

Overlay ROIs

In [ ]:
overlay_1 = overlay_colorized_rois(background_1, CR1)
overlay_2 = overlay_colorized_rois(background_2, CR2)

Generate Pixel Pairs

In [ ]:
pp_0 = generate_pixel_pairs(stat, ColorVideo.neuron_subsets[0])
pp_1 = generate_pixel_pairs(stat, ColorVideo.neuron_subsets[1])

Merge

In [ ]:
merge_1 = merge_background(ColorVideo.background.copy(), overlay_1, pp_0)
merge_2 = merge_background(ColorVideo.background.copy(), overlay_2, pp_1)
merge_3 = merge_background(ColorVideo.background.copy(), merge_1, pp_0)
merge_4 = merge_background(merge_3.copy(), merge_2, pp_1)

Save

In [ ]:
save_video(merge_1, "C:\\Users\\YUSTE\\Desktop\\test_A.mp4", fps=10)
save_video(merge_2, "C:\\Users\\YUSTE\\Desktop\\test_B.mp4", fps=10)
save_video(merge_3, "C:\\Users\\YUSTE\\Desktop\\test_C.mp4", fps=10)
save_video(merge_4, "C:\\Users\\YUSTE\\Desktop\\test_D.mp4", fps=10)